In [1]:
import tushare as ts
import easytrader as trade
from easytrader import helpers
from easytrader import webtrader 
import pandas as pd
import datetime as dt
from mercury import utils
from mercury import uqer
import os,shutil
pd.options.display.max_columns = 100
pd.options.display.max_rows = 300

#uquer token ='5fa4680adf1c779e93f1d90880b4e2f55a52a7704f2408565ca8c12eebba0469'
sess = uqer.Client(token='5fa4680adf1c779e93f1d90880b4e2f55a52a7704f2408565ca8c12eebba0469')

user = trade.use('xq')
user.prepare('./xq_new_issue352639.json')

## 获取资产情况 
#balance = user.balance
## 获取股票持仓
#position = user.position
## 获取当日委托
#enturst = user.entrust
## 查询当日成交
#user.current_deal
## 查询最近30天的交割单
# user.exchangebill   
## 指定查询时间段, 日期格式为 "20160214"
# user.get_exchangebill('开始日期', '截止日期')   

# user.buy('162411', price=0.55, amount=100) ## 买入委托
# user.sell('162411', price=0.55, amount=100) ## 卖出委托
# user.cancel_entrust('委托单号', '股票代码')  ##撤单

#helpers.get_today_ipo_data() ##当日申购新股
# user.get_ipo_limit('申购代码') ##申购限额

##雪球组合调仓
#user.adjust_weight('000001', 10)

2016-12-06 10:11:01,459 [DEBUG] xqtrader.py 54: login status: SUCCESS


In [2]:
#获取优矿当前持仓
max_pos = 20 

today = dt.date.today() 
filename = 'new_issue_'+today.strftime('%Y%m%d')+'.csv'
sess.download_data(filename)
uqer = pd.DataFrame()
uqer = uqer.from_csv(filename)
if len(uqer) > 0 : 
    uqer = uqer.reset_index()
    uqer = uqer.set_index(uqer.secID.apply(lambda x:x[0:6]).values)
    print(uqer)  

    # 获取当前需求持仓
    mkv = pd.DataFrame(user.balance)
    cash_pct = 100 * mkv.current_balance.values[0] / mkv.asset_balance.values[0]
    pos = pd.DataFrame(user.position)
    if len(pos) > 0 :   
        pos = pos.set_index(pos.stock_code.apply(lambda x:x[2:8]).values)
        pos['c_pct'] = 100.0 * pos.market_value / mkv.asset_balance.values[0] 


    # 开始调仓, 卖出不在类表中的, 调整股票池中的比例
    for s in pos.index:
        if not s in uqer.index:
            print('调出: ', pos.loc[s,'stock_name'])
            try : 
                user.adjust_weight(s,0.0)  
            except webtrader.TradeError as e: 
                print(s,'调整失败',e)
    for s in uqer.index:
        if not s in pos.index:
            pct = min(max_pos,cash_pct)
            print('调入: ', s, pct)
            try : 
                user.adjust_weight(s,pct)  
            except webtrader.TradeError as e: 
                print(s,'调整失败',e)

shutil.copyfile(filename,".\\data\\" + filename)    
os.remove(".\\"+filename)




Start download new_issue_20161206.csv

Done download new_issue_20161206.csv
        ticker  ticker.1        secID secShortName    listDate  totalShares  \
002813    2813      2813  002813.XSHE         路畅科技  2016-10-12    120000000   
300548  300548    300548  300548.XSHE         博创科技  2016-10-12     82670000   
300555  300555    300555  300555.XSHE         路通视信  2016-10-18     80000000   
300553  300553    300553  300553.XSHE         集智股份  2016-10-21     48000000   
603859  603859    603859  603859.XSHG         能科股份  2016-10-21    113560000   
603258  603258    603258  603258.XSHG         电魂网络  2016-10-26    240000000   
603716  603716    603716  603716.XSHG          塞力斯  2016-10-31     50940000   
002818    2818      2818  002818.XSHE          富森美  2016-11-09    440000000   

        nonrestFloatShares     endDate  
002813            30000000  2016-09-30  
300548            20670000  2016-09-30  
300555            20000000  2016-09-30  
300553            12000000  2016-09-30  
603859

2016-12-06 10:11:10,915 [DEBUG] xqtrader.py 304: 调仓比例:0.000000, 剩余持仓 :23.110000
2016-12-06 10:11:11,080 [DEBUG] xqtrader.py 319: 调仓 五洲新春: 持仓比例0
2016-12-06 10:11:11,081 [DEBUG] xqtrader.py 326: 调仓成功 五洲新春: 持仓比例0


 五洲新春
调入: 

2016-12-06 10:11:11,188 [DEBUG] xqtrader.py 304: 调仓比例:20.000000, 剩余持仓 :3.110000
2016-12-06 10:11:11,312 [DEBUG] xqtrader.py 319: 调仓 集智股份: 持仓比例20
2016-12-06 10:11:11,312 [DEBUG] xqtrader.py 326: 调仓成功 集智股份: 持仓比例20


 300553 20
调入: 

2016-12-06 10:11:11,436 [DEBUG] xqtrader.py 304: 调仓比例:20.000000, 剩余持仓 :-16.890000
2016-12-06 10:11:11,656 [DEBUG] xqtrader.py 319: 调仓 能科股份: 持仓比例20
2016-12-06 10:11:11,656 [DEBUG] xqtrader.py 326: 调仓成功 能科股份: 持仓比例20


 603859 20
调入: 

2016-12-06 10:11:11,779 [DEBUG] xqtrader.py 304: 调仓比例:20.000000, 剩余持仓 :-20.000000
2016-12-06 10:11:11,900 [DEBUG] xqtrader.py 319: 调仓 富森美: 持仓比例20
2016-12-06 10:11:11,903 [DEBUG] xqtrader.py 326: 调仓成功 富森美: 持仓比例20


 002818 20
